## Описание проекта  

Нужно создать систему, которая могла бы оценить риск ДТП по выбранному маршруту движения. Под риском понимается вероятность ДТП с любым повреждением транспортного средства. Как только водитель забронировал автомобиль, сел за руль и выбрал маршрут, система должна оценить уровень риска. Если уровень риска высок, водитель увидит предупреждение и рекомендации по маршруту.  

Идея создания такой системы находится в стадии предварительного обсуждения и проработки. Чёткого алгоритма работы и подобных решений на рынке ещё не существует. Текущая задача — понять, возможно ли предсказывать ДТП, опираясь на исторические данные одного из регионов.

Импортируем библиотеки

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import catboost as cb

from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from catboost import MetricVisualizer
from catboost import  Pool

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import recall_score, precision_score

from sqlalchemy import create_engine

## Подключитесь к базе. Загрузите таблицы sql

In [ ]:
db_config = {
'user': 'praktikum_student', # имя пользователя,
'pwd': 'Sdf4$2;d-d30pp', # пароль,
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения,
'db': 'data-science-vehicle-db' # название базы данных,
}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

In [ ]:
engine = create_engine(connection_string)

## Проведите первичное исследование таблиц

Загрузим все значения таблицы **collisions**

In [ ]:
query = '''
SELECT *
FROM collisions;
'''

df_collisions = pd.read_sql_query(query, con=engine)

Выведем на экран первые 3 строки полученного датафрейма, и выведем информацию о DataFrame, включая тип столбца, ненулевые значения.

In [ ]:
display(df_collisions.head(3))
display(df_collisions.info())

,case_id,county_city_location,county_location,distance,direction,intersection,weather_1,location_type,collision_damage,party_count,primary_collision_factor,pcf_violation_category,type_of_collision,motor_vehicle_involved_with,road_surface,road_condition_1,lighting,control_device,collision_date,collision_time
0,4083072,1942,los angeles,528.0,north,0.0,cloudy,highway,small damage,2,vehicle code violation,unsafe lane change,sideswipe,other motor vehicle,wet,normal,daylight,none,2009-01-22,07:25:00
1,4083075,4313,santa clara,0.0,None,1.0,clear,None,small damage,1,vehicle code violation,improper passing,hit object,fixed object,dry,normal,dark with street lights,functioning,2009-01-03,02:26:00
2,4083073,0109,alameda,0.0,None,1.0,clear,None,scratch,2,vehicle code violation,improper turning,broadside,other motor vehicle,dry,normal,dark with street lights,functioning,2009-01-11,03:32:00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400000 entries, 0 to 1399999
Data columns (total 20 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   case_id                      1400000 non-null  object 
 1   county_city_location         1400000 non-null  object 
 2   county_location              1400000 non-null  object 
 3   distance                     1400000 non-null  float64
 4   direction                    1059358 non-null  object 
 5   intersection                 1387781 non-null  float64
 6   weather_1                    1392741 non-null  object 
 7   location_type                518779 non-null   object 
 8   collision_damage             1400000 non-null  object 
 9   party_count                  1400000 non-null  int64  
 10  primary_collision_factor     1391834 non-null  object 
 11  pcf_violation_category       1372046 non-null  object 
 12  type_of_collision            1388176 non-n

None

Так же посмотрим что в таблице **parties**

In [ ]:
query = '''
SELECT *
FROM parties
'''

df_parties = pd.read_sql_query(query, con=engine)

Выведем на экран первые 3 строки полученного датафрейма, и выведем информацию о DataFrame, включая тип столбца, ненулевые значения.

In [ ]:
display(df_parties.head(3))
display(df_parties.info())

Посмотрим на таблицу **vehicles**

In [ ]:
query = '''
SELECT *
FROM vehicles
'''

df_vehicless = pd.read_sql_query(query, con=engine)

In [ ]:
display(df_vehicless.head(3))
display(df_vehicless.info())

### Вывод:  

Из начального задания нам известно, что:  
 - collisions (информация о происшествиях);
 - Parties (описание участников происшествия);
 - Vehicles (Описание автомобиля)  

Все ли таблицы имеют набор данных.  
Количество таблиц соответствует условию задачи.  
Имеется общий ключ для связи таблиц case_id и party_number.


##  Проведите статистический анализ факторов ДТП

Выясним, в какие месяцы происходит наибольшее количество аварий. Проанализируем весь период наблюдений (таблица **collisions**)

In [ ]:
query = '''
SELECT EXTRACT(YEAR FROM collision_date) AS year_collision,
       EXTRACT(MONTH FROM collision_date) AS month_collision,
       COUNT(case_id) AS count_collision
FROM collisions
GROUP BY 1, 2
ORDER BY 1, 3 DESC
'''
collisions_count = pd.read_sql_query(query, con=engine)

Построим график

In [ ]:
fig, ax = plt.subplots(figsize=(18, 8))
sns.barplot(data = collisions_count, x='month_collision', y='count_collision', hue='year_collision')
plt.title('Количество аварий в месяц')
plt.xlabel('Месяц')
plt.ylabel('Количество аварий')
plt.show()

Вывод: наибольшее количество ДТП за 2009-20011 приходится на октябрь, декабрь и март месяц. В 2012 году с июня месяца обнаружено снижение количества ДТП с 25000-33000 до 200-3000. Аномально маленькие значения по годам и месяцам можно отнести к не полноте информации в базе данных, так как другое обоснование найти трудно.

Скоро состоится первое совещание рабочей группы. Чтобы обсуждение было конструктивным, каждый сотрудник должен понимать данные. Для этого мы создадим подходящие аналитические задачи и поручим их решение коллегам.  

Опишем их:
 - провести анализ серьёзности повреждений транспортного средства, исходя из состояния дороги в момент ДТП (связать collisions и parties);
 - автомобили с каким типом кузова чаще получают серьезные повреждения при ДТП
 - проанадизировать самые частые причины ДТП (таблица parties);
 - изучить трезвость участников ДТП и оценить его влияние на виновность участника;
 - изучить серьезность повреждений ТС в зависимости от трезвости участника;
 - провести анализ зависимости ДТП и погоды;
 - выявить типы КПП и возраст автомобилей, и ценить их влияние на ДТП.

По заданию, нам необходимо произвести порядок решения для двух задач из списка. Обязательное условие — решение этих задач должно включать связь не менее 2-х таблиц.

### Проведем анализ серьёзности повреждений транспортного средства, исходя из состояния дороги в момент ДТП

In [ ]:
query = '''
select distinct road_surface, collision_damage,
COUNT(collision_damage) over (partition by road_surface order by collision_damage) as col_count
from collisions c
where case_id in (select distinct case_id
                  from parties p
                  where party_type = 'car')
order by road_surface, col_count'''

col_with_weather = pd.read_sql_query(query, con=engine)
col_with_weather.head(10)

Построим график

In [ ]:
f, ax = plt.subplots(figsize=(18, 8))
sns.barplot(data =col_with_weather,
            x='road_surface',
            y = 'col_count',
            hue  = 'collision_damage',
            palette = "Set2",
            alpha=0.7)
ax.legend(ncol=1, loc="upper right", frameon=True)
ax.set(xlabel="Количество",
       ylabel="Дата")
#plt.xticks(rotation=90)
plt.title(" Анализ серьёзности повреждений транспортного средства, исходя из состояния дороги в момент ДТП")
plt.show()

#### Вывод:
 - Наибольшее количество ДТП происходит на сухой дороге. Также много ДТП происходит на сырой дороге. Больше всего повреждений - это небольшие повреждения и царапины, как на сухой дороге, так и на мокрой.

### Проведем анализ автомобили с каким типом кузова чаще получают серьезные повреждения при ДТП

In [ ]:
sql_query = '''
select *,
col_count/SUM(col_count) over (partition by vehicle_type) as prcnt
from (
select distinct collision_damage, vehicle_type,
count(collision_damage) over(partition by vehicle_type order by collision_damage) as col_count
from
(select case_id , collision_damage
from collisions) c
join
(select case_id, vehicle_type
from vehicles) v  on c.case_id = v.case_id) t
'''
car_crash = pd.read_sql_query(sql_query, con=engine)
car_crash = car_crash.fillna('unknown') # заполним пропуски
car_crash.head(10)

Построим график

In [ ]:
f, ax = plt.subplots(figsize=(18, 8))
sns.barplot(data =car_crash,
            x='vehicle_type',
            y = 'col_count',
            hue  = 'collision_damage',
            palette = "Set2",
            alpha=0.7)
ax.legend(ncol=1, loc="upper right", frameon=True)
ax.set(xlabel="Количество",
       ylabel="Дата")
plt.title("Количество проишествий по месяцам")
plt.show()

Вывод: в количественном отношении во всех категориях кроме легких повреждений лидируют машины с типом кузова купе.

## Создайте модель для оценки водительского риска

Для создания модели нам понадобятся следующие признаки:

таблица **collisions**:  
 - `intersection` (является ли место происшествия перекрестком)
 - `weather_1` (погода)
 - `road_surface` (состояние дороги)
 - `lighting` (освещение)
 - `location_type` (тип дороги)
 - `road_condition_1` (состояние дорожного покрытия)  

таблица **parties**:
 - `at_fault` (виновность участника) - целевой признак
 - `party_sobriety` (трезвость участника
 - `party_drug_physical` (состояние участника: физическое или с учётом принятых лекарств)
 - `cellphone_in_use` (наличие телефона в автомобиле - возможности разговаривать по громкой связи)  

таблица **vehicles**:
 - `vehicle_type` (тип кузова)
 - `vehicle_transmission` (тип КПП)
 - `vehicle_age` (возраст автомобиля)  

Все эти признаки важны, так как отражают и объективные причины ДТП (состояние дороги и транспортного средства), и субъективные - состояние участника ДТП.  

По заданию, подготовим набор данных на основе первичного предположения заказчика:
 - выберем тип виновника — только машина (car)
 - возьмите случаи, когда ДТП привело к любым значимым повреждениям автомобиля любого из участников — все, кроме типа SCRATCH (царапина)
 - для моделирования возьмите данные только за 2012 год

In [ ]:
query = '''
with
c AS (SELECT case_id,
             intersection,
             weather_1,
             road_surface,
             lighting,
             location_type,
             road_condition_1
      FROM collisions
      WHERE EXTRACT(YEAR FROM collision_date) = 2012
            AND collision_damage != 'scratch'),
p AS (SELECT case_id,
             at_fault,
             party_sobriety,
             party_drug_physical,
             cellphone_in_use
      FROM parties
      WHERE party_type = 'car'),

v AS (SELECT case_id,
             vehicle_type,
             vehicle_transmission,
             vehicle_age
       FROM vehicles),

c_i as (SELECT case_id
        FROM case_ids)

SELECT  p.at_fault,
        p.party_sobriety,
        p.party_drug_physical,
        p.cellphone_in_use,
        c.intersection,
        c.weather_1,
        c.road_surface,
        c.lighting,
        c.location_type,
        c.road_condition_1,
        v.vehicle_type,
        v.vehicle_transmission,
        v.vehicle_age
FROM c_i
JOIN c on c_i.case_id = c.case_id
JOIN p on c_i.case_id = p.case_id
JOIN v on c_i.case_id = v.case_id
'''

df = pd.read_sql_query(query, con=engine)

Посмотрим на записи в датафрейме и информацию о нем

In [ ]:
display(df.head(10))
display(df.info())

Посмотрим на пропуски

In [ ]:
df.isna().sum()

In [ ]:
df.isna().mean()

Столбец **party_drug_physical** мог бы быть полезен, но в нем отсутствует более 90% данных, поэтому особой ценности он не представляет, удалим его

In [ ]:
df = df.drop('party_drug_physical', axis=1)

Посмотрим распределение возраста автомобилей

In [ ]:
df['vehicle_age'].value_counts().plot(figsize=(18,8), kind='bar')
plt.title('Гистограмма распределения возраста автомобиля')
plt.xlabel('Возраст автомобиля')
plt.ylabel('Количество автомобилей')
plt.legend([])
plt.show()

Возраст автомобиля заменим значения > 15 на 15

In [ ]:
df.loc[df['vehicle_age'] > 15, 'vehicle_age'] = 15

In [ ]:
df.describe()

Заменим пропуски в числовых данных на значение медианы

In [ ]:
df = df.fillna(df[{'vehicle_age','cellphone_in_use','intersection'}].median())
df.isna().sum()

Посмотрим на распределение значений **столбца location_type**

In [ ]:
df['location_type'].value_counts().plot(figsize=(6,4), kind='bar')
plt.title('Гистограмма распределения типа дороги')
plt.xlabel('Тип дороги')
plt.ylabel('Количество')
plt.legend([])
plt.show()

Посмотрим на распределение значений столбца **party_sobriety**

In [ ]:
df['party_sobriety'].value_counts().plot(figsize=(6,4), kind='bar')
plt.title('Гистограмма распределения трезвости участника')
plt.xlabel('Трезвость участника')
plt.ylabel('Количество')
plt.legend([])
plt.show()

Заменим пропуски в столбце **party_sobriety**, **weather_1**, **road_surface**, **lighting**, **road_condition_1**, **vehicle_transmission** значениями с такой же вероятностью

In [ ]:
for i in df[['location_type', 'party_sobriety', 'weather_1', 'road_surface', 'lighting', 'road_condition_1', 'vehicle_transmission']]:
    mask = df[i].isna()
    p = df[i].value_counts() / len(df[i].dropna())
    df.loc[mask, i] = np.random.choice(p.index.to_list(), size=mask.sum(), p=p.to_list())

df.isna().sum()

### Вывод:
 - удалили столбец party_drug_physical, так как пропусков более 90%, для модели не даст никакой значимости;
 - в числовых признаках 'vehicle_age','cellphone_in_use','intersection' заменили пропуски медианой;
 - пропуски в признаках **party_sobriety**, **weather_1**, **road_surface**, **lighting**, **road_condition_1**, **vehicle_transmission** заменили значениями с такой же вероятностью

Проверим полученный датасет на дубликаты, подсчитаем количество дубликатов для каждой уникальной строки в DataFrame. В столбце размера отображается количество дубликатов для каждой уникальной строки.

In [ ]:
df.groupby(df.columns.tolist(), as_index= False ).size()

Выведем новую информацию датафрейма первые 5 строк

In [ ]:
df.head()

Создадим датафрейм для кодирования и масштабирования

In [ ]:
df_encoder = df.copy()

Создадим признаки и целевой признак

In [ ]:
features = df.drop('at_fault', axis=1)
target = df['at_fault']

features_encoder = df_encoder.drop('at_fault', axis=1)
target_encoder = df_encoder['at_fault']

Посмотрим на распределение целевого призака

In [ ]:
target.value_counts().plot(figsize=(6,4), kind='bar')
plt.title('Распределение классов целевого признака')
plt.xlabel('Класс признака')
plt.ylabel('Количество')
plt.legend([])
plt.show()

Можно сказать классы сбалансированы

Разделим на обучающую, валидационную и тестовую выборки для СatBoost

In [ ]:
features_train, features_other, target_train, target_other = train_test_split(features.copy(), target.copy(), test_size=0.4,
                                                                              shuffle=True, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_other.copy(), target_other.copy(), test_size=0.5,
                                                                            shuffle=True, random_state=12345)

Проверим размер выборок

In [ ]:
display(features_train.shape, features_valid.shape, features_test.shape)
display(target_train.shape, target_valid.shape, target_test.shape)

Разделим на обучающую, валидационную и тестовую выборки для моделей DecisionTree и RandomForest

In [ ]:
features_train_encoder, features_other_encoder, target_train_encoder, target_other_encoder = train_test_split(
    features_encoder.copy(), target_encoder.copy(), test_size=0.4, shuffle=True, random_state=12345)

features_valid_encoder, features_test_encoder, target_valid_encoder, target_test_encoder = train_test_split(
    features_other_encoder.copy(), target_other_encoder.copy(), test_size=0.5, shuffle=True, random_state=12345)

Проверим размер выборок

In [ ]:
display(features_train_encoder.shape, features_valid_encoder.shape, features_test_encoder.shape)
display(target_train_encoder.shape, target_valid_encoder.shape, target_test_encoder.shape)

### Масштабируем числовые признаки

In [ ]:
numeric = ['vehicle_age']

scaler = StandardScaler()
scaler.fit(features_train_encoder[numeric])

features_train_encoder[numeric] = scaler.transform(features_train_encoder[numeric])
features_valid_encoder[numeric] = scaler.transform(features_valid_encoder[numeric])
features_test_encoder[numeric] = scaler.transform(features_test_encoder[numeric])

### Кодирование категориальных признаков с помощью OHE

In [ ]:
categorial = ['party_sobriety', 'weather_1', 'road_surface', 'lighting', 'location_type', 'road_condition_1',
              'vehicle_type', 'vehicle_transmission']

encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
encoder.fit(features_train_encoder[categorial])

features_train_encoder_ohe = pd.DataFrame(encoder.transform(features_train_encoder[categorial]), columns = encoder.get_feature_names(categorial))
features_valid_encoder_ohe = pd.DataFrame(encoder.transform(features_valid_encoder[categorial]), columns = encoder.get_feature_names(categorial))
features_test_encoder_ohe = pd.DataFrame(encoder.transform(features_test_encoder[categorial]), columns = encoder.get_feature_names(categorial))

Удаляем столбцы с категориальными признаками

In [ ]:
features_train_encoder = features_train_encoder.drop(categorial, axis=1)
features_valid_encoder = features_valid_encoder.drop(categorial, axis=1)
features_test_encoder = features_test_encoder.drop(categorial, axis=1)

Объединяем таблицы

In [ ]:
features_train_encoder = pd.concat([features_train_encoder_ohe.reset_index(drop=True), features_train_encoder.reset_index(drop=True)], axis=1, ignore_index=False)
features_valid_encoder = pd.concat([features_valid_encoder_ohe.reset_index(drop=True), features_valid_encoder.reset_index(drop=True)], axis=1, ignore_index=False)
features_test_encoder = pd.concat([features_test_encoder_ohe.reset_index(drop=True), features_test_encoder.reset_index(drop=True)], axis=1, ignore_index=False)

Проверим кодирование и масштабирование признаков

In [ ]:
features_train_encoder.head()

В качестве метрики будем использовать roc-auc, потому что нам важно, чтобы модель с большей степенью уверенности прогнозировала классы.

### DecisionTreeClassifier

Зададим гиперпараметры модели

In [ ]:
decision_tree_params = {
    'max_depth': range(1, 20, 2),
    'min_samples_split': [5],
    'min_samples_leaf': [25],
    'class_weight':['balanced']
}

dt_gs = GridSearchCV(DecisionTreeClassifier(random_state=5),
                         decision_tree_params, n_jobs=-1, cv=5, scoring='roc_auc')
dt_f1 = dt_gs.fit(features_train_encoder, target_train_encoder)

Найдем лучшие гиперпараметры

In [ ]:
dt_f1.best_params_

Предсказания модели с лучшими гиперпараметрами

In [ ]:
pred_dt = dt_f1.best_estimator_.predict(features_valid_encoder)

print('F1 на валидационной выборке:', f1_score(pred_dt, target_valid_encoder))
print('ROC_AUC на валидационной выборке:', roc_auc_score(pred_dt, target_valid_encoder))

### RandomForestClassifier

Зададим гиперпараметры модели

In [ ]:
param_grid = {
    'n_estimators': range(80, 101, 5),
    'max_depth': range(1, 11, 2),
    'class_weight':['balanced']
}

rf = RandomForestClassifier(random_state=5)
rf_grid = GridSearchCV(rf, param_grid, cv=5, n_jobs=-1, verbose=2)

Обучим модель

In [ ]:
rf_grid.fit(features_train_encoder, target_train_encoder)

Найдем лучшие гиперпараметры

In [ ]:
rf_grid.best_params_

Предсказания модели с лучшими гиперпараметрами

In [ ]:
pred_rf = rf_grid.best_estimator_.predict(features_valid_encoder)
print('F1 на валидационной выборке:', f1_score(pred_rf, target_valid_encoder))
print('ROC_AUC на валидационной выборке:', roc_auc_score(pred_rf, target_valid_encoder))

### CatBoostClassifier

In [ ]:
model_cat = cb.CatBoostClassifier(loss_function='Logloss', eval_metric='AUC')

Зададим категориальные признаки

In [ ]:
cat_features = ['party_sobriety', 'weather_1', 'road_surface', 'lighting', 'location_type', 'road_condition_1',
              'vehicle_type', 'vehicle_transmission']

Зададим гиперпараметры модели

In [ ]:
grid = {'iterations': [100, 120],
        'learning_rate': [0.001, 0.002],
        'depth': range(6, 9)
       }

In [ ]:
catboost_grid = model_cat.grid_search(grid, Pool(features_train, target_train, cat_features=cat_features), cv=3,
                                  verbose=50)

Найдем лучшие гиперпараметры

In [ ]:
catboost_grid['params']

Зададим гиперпараметры модели

In [ ]:
final_cat = cb.CatBoostClassifier(**catboost_grid['params'],
                              eval_metric='AUC', verbose=100, train_dir='learning_rate_0.01')

Обучим модель

In [ ]:
final_cat.fit(Pool(features_train, target_train, cat_features=cat_features))

Предсказания модели

In [ ]:
pred_cat = final_cat.predict(features_valid)

print('F1 на валидационной выборке:', f1_score(pred_cat, target_valid))
print('ROC_AUC на валидационной выборке:', roc_auc_score(pred_cat, target_valid))

Построим матрицу ошибок для модели CatBoost

In [ ]:
matrix = confusion_matrix(target_valid, pred_cat)
cm_display = ConfusionMatrixDisplay(matrix).plot()

Построим матрицу ошибок для модели Randomforest

In [ ]:
matrix_rf = confusion_matrix(target_valid_encoder, pred_rf)
cm_display_rf = ConfusionMatrixDisplay(matrix_rf).plot()

Построим матрицу ошибок для модели DecisionTreeClassifier

In [ ]:
matrix_dt = confusion_matrix(target_valid_encoder, pred_dt)
cm_display_dt = ConfusionMatrixDisplay(matrix_dt).plot()

Построим сводную таблицу

In [ ]:
data = {'F1': [0.422, 0.405, 0.341],
        'AUC_ROC': [0.643, 0.655, 0.709]}

table = pd.DataFrame(data, index=['DecisionTree', 'RandomForest', 'CatBoost'])
table.sort_values('AUC_ROC', ascending=False)

Наилучшей моделью оказалась модель CatBoost, которая показала значение ROC-AUC 0.709, количество FN - 8707, FP - 466, при этом модель RandomForest показала количество ложноположительных ответов меньше - 8022, FP - 1149.

Тестирование модели CatBoost. Объединим обучающую и валидационные выбоки

In [ ]:
features_train_valid = pd.concat([features_train, features_valid])
target_train_valid = pd.concat([target_train, target_valid])

Обучим модель

In [ ]:
final_cat.fit(Pool(features_train_valid, target_train_valid, cat_features=cat_features))

Предсказания модели

In [ ]:
%%time
pred_cat_test = final_cat.predict(features_test)

print('F1 на валидационной выборке:', f1_score(pred_cat_test, target_test))
print('ROC_AUC на валидационной выборке:', roc_auc_score(pred_cat_test, target_test))

In [ ]:
matrix = confusion_matrix(target_test, pred_cat_test)
cm_display = ConfusionMatrixDisplay(matrix).plot()

Модель CatBoost на тестовой выборке показала почти такие же результаты, как и на валидационной. ROC-AUC 0.709, количество FN ответов 8686, FP - 476

Проведем небольшой анализ качества модели. Для модели CatBoostClassifier построим ROC_AUC кривую, полноту и точность.

In [ ]:
probabilities_valid = final_cat.predict_proba(features_train)
probabilities_one_valid = probabilities_valid[:, 1]

def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

fpr, tpr, thresholds = roc_curve(target_train, probabilities_one_valid)
print(roc_auc_score(pred_cat, target_valid))
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print("Threshold value is:", optimal_threshold)
plot_roc_curve(fpr, tpr)

In [ ]:
roc_auc_score(pred_cat_test, target_test)

In [ ]:
print('Recall', recall_score(pred_cat, target_valid>=optimal_threshold))
print('Precision', precision_score(pred_cat, target_valid>=optimal_threshold))

precision, recall, _ = precision_recall_curve(pred_cat, target_valid)

disp = PrecisionRecallDisplay(precision=precision, recall=recall)
disp.plot()
plt.scatter(recall_score(pred_cat, target_valid>=optimal_threshold),
            precision_score(pred_cat, target_valid>=optimal_threshold))
plt.title("precision_recall_curve")
plt.show()

Выведем на экран полноту (recall). Она выявляет, какую долю положительных среди всех ответов выделила модель. Обычно они на вес золота, и важно понимать, как хорошо модель их находит.

In [ ]:
print('Полнота DecisionTreeClassifier:', recall_score(pred_dt, target_valid_encoder))
print('Полнота RandomForestClassifier:', recall_score(pred_rf, target_valid_encoder))
print('Полнота CatBoostClassifier:', recall_score(pred_cat, target_valid))

Вывод: значение recall не далеко от единицы: модель не очень хорошо ищет положительные объекты.

Посмотрим на еще одну метрику для оценки качества прогноза целевого класса — точность (precision).

In [ ]:
print('Точность DecisionTreeClassifier:', precision_score(pred_dt, target_valid_encoder))
print('Точность RandomForestClassifier:', precision_score(pred_rf, target_valid_encoder))
print('Точность CatBoostClassifier:', precision_score(pred_cat, target_valid))

Вывод: Точность определяет, как много отрицательных ответов нашла модель, пока искала положительные. Чем больше отрицательных, тем ниже точность.

## Проведите анализ важности факторов ДТП

In [ ]:
importances = final_cat.feature_importances_
feature_list = list(features_train_valid.columns)
feature_results = pd.DataFrame({'feature': feature_list,'importance': importances})
feature_results = feature_results.sort_values('importance',ascending = False).reset_index(drop=True)

feature_results.head()

In [ ]:
indices = np.argsort(importances)

fig, ax = plt.subplots(figsize=(18, 8))
ax.barh(range(len(importances)), importances[indices])
ax.set_yticks(range(len(importances)))
_ = ax.set_yticklabels(np.array(features_train_valid.columns)[indices])

In [ ]:
group_party_sobriety = df[df['at_fault'] == 1].groupby('party_sobriety')['at_fault'].count().plot(kind='bar', figsize=(18, 8))

In [ ]:
y_sobriety = df[df['at_fault'] == 0].groupby('party_sobriety')['at_fault'].count().plot(kind='bar', figsize=(18, 8))

## Выводы

По заданию, нам нужно было создать систему, которая могла бы оценить риск ДТП по выбранному маршруту движения.  

Из графиков видим, что самый важный фактор ДТП — уровень трезвости виновника **party_sobriety**. Из таблицы исходных данных известно, что есть несколько уровней трезвости. Благодоря столбчатой диаграмме, которая отражает зависимость числа ДТП от уровня трезвости, можно сделать вывод, что в основном ДТП происходят, когда участник трезв, но вероятность виновности в ДТП выше, когда участник пьян.  

Для решения проблемы необходимо оборудовать автомобиль анализатором на трезвость водителя. Измерение состояния при посадке сделать обязательным условием допуска к управлению автомобилем. Чтобы убедиться, что тест проходит именно водитель - добавить камеру, направленную на водительское кресло.  

Наилучшей оказалась модель CatBoost с гиперпараметрами:
 - depth - 6,
 - iterations - 100,
 - learning_rate - 0.002,  

которая на тестовой выборке показала результаты:
 - ROC-AUC 0.709,
 - количество FN ответов 8686,
 - FP - 476.  

Для улучшения модели можно добавить такие признаки, как стаж вождения, количество штрафов за нарушение ПДД, попадал ли ранее водитель в ДТП.